# 📊 Análise de Dados PNCP - Portal Nacional de Contratações Públicas

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/franklinbaldo/baliza/blob/main/notebooks/analise_pncp_colab.ipynb)

Este notebook fornece acesso direto aos dados de contratações públicas preservados no **Internet Archive** pelo projeto [Baliza](https://github.com/franklinbaldo/baliza).

## 🎯 O que você pode fazer:
- ✅ Analisar **milhões de contratos públicos** desde 2021
- ✅ Criar **visualizações interativas** com Plotly
- ✅ Detectar **padrões suspeitos** em contratações
- ✅ Análises **geográficas e temporais**
- ✅ Comparar **órgãos e fornecedores**
- ✅ Exportar dados para **pesquisa acadêmica**

## 📚 Fonte dos Dados:
- **PNCP**: Portal Nacional de Contratações Públicas
- **Preservação**: Internet Archive (permanente)
- **Formato**: Parquet (otimizado para análise)
- **Atualização**: Diária via GitHub Actions

---
**⚠️ Importante**: Este notebook funciona 100% no Google Colab sem necessidade de configuração local!

## 🔧 Configuração do Ambiente

Instala as dependências necessárias e configura o acesso aos dados do Internet Archive.

In [ ]:
# Instalar dependências necessárias
!pip install -q pandas plotly duckdb requests internetarchive seaborn folium

print("✅ Dependências instaladas com sucesso!")

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import duckdb
import requests
import json
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configurações do Plotly para melhor visualização
import plotly.io as pio
pio.templates.default = "plotly_white"

print("📦 Bibliotecas importadas com sucesso!")
print(f"🐼 Pandas: {pd.__version__}")
print(f"🦆 DuckDB: {duckdb.__version__}")

## 🌐 Acesso aos Dados do Internet Archive

Vamos descobrir e carregar os dados PNCP preservados no Internet Archive.

In [ ]:
def discover_baliza_data():
    """Descobre dados Baliza disponíveis no Internet Archive."""
    print("🔍 Descobrindo dados Baliza no Internet Archive...")
    
    # API do Internet Archive para buscar itens Baliza
    search_url = "https://archive.org/advancedsearch.php"
    params = {
        'q': 'title:baliza-* AND mediatype:data',
        'fl': 'identifier,title,description,date,item_size',
        'sort[]': 'date desc',
        'rows': 50,
        'page': 1,
        'output': 'json'
    }
    
    try:
        response = requests.get(search_url, params=params, timeout=30)
        response.raise_for_status()
        data = response.json()
        
        items = data.get('response', {}).get('docs', [])
        
        print(f"📊 Encontrados {len(items)} itens Baliza no Internet Archive")
        
        # Mostrar os 5 mais recentes
        if items:
            print("\n📋 Itens mais recentes:")
            for i, item in enumerate(items[:5]):
                identifier = item.get('identifier', 'N/A')
                title = item.get('title', 'N/A')
                date = item.get('date', 'N/A')
                size_mb = int(item.get('item_size', 0)) / (1024*1024) if item.get('item_size') else 0
                print(f"  {i+1}. {identifier} ({date}) - {size_mb:.1f} MB")
        
        return items
        
    except Exception as e:
        print(f"❌ Erro ao descobrir dados: {e}")
        return []

# Descobrir dados disponíveis
baliza_items = discover_baliza_data()

In [ ]:
def get_parquet_files_from_item(identifier):
    """Obtém lista de arquivos Parquet de um item do IA."""
    files_url = f"https://archive.org/metadata/{identifier}"
    
    try:
        response = requests.get(files_url, timeout=30)
        response.raise_for_status()
        metadata = response.json()
        
        parquet_files = []
        files = metadata.get('files', [])
        
        for file_info in files:
            if file_info.get('name', '').endswith('.parquet'):
                parquet_files.append({
                    'name': file_info['name'],
                    'size': int(file_info.get('size', 0)),
                    'url': f"https://archive.org/download/{identifier}/{file_info['name']}"
                })
        
        return parquet_files
        
    except Exception as e:
        print(f"❌ Erro ao obter arquivos de {identifier}: {e}")
        return []

# Obter arquivos Parquet do item mais recente
if baliza_items:
    latest_item = baliza_items[0]['identifier']
    print(f"\n📁 Buscando arquivos Parquet em: {latest_item}")
    
    parquet_files = get_parquet_files_from_item(latest_item)
    
    if parquet_files:
        print(f"\n✅ Encontrados {len(parquet_files)} arquivos Parquet:")
        for i, file_info in enumerate(parquet_files[:5]):
            size_mb = file_info['size'] / (1024*1024)
            print(f"  {i+1}. {file_info['name']} ({size_mb:.1f} MB)")
    else:
        print("⚠️ Nenhum arquivo Parquet encontrado")
        parquet_files = []
else:
    print("⚠️ Nenhum item Baliza encontrado")
    parquet_files = []

## 📊 Carregamento dos Dados

Carrega uma amostra dos dados para análise. Para datasets grandes, usamos apenas parte dos dados para performance.

In [ ]:
def load_sample_data(max_files=3, sample_size=100000):
    """Carrega amostra dos dados PNCP do Internet Archive."""
    
    if not parquet_files:
        print("❌ Nenhum arquivo Parquet disponível")
        return pd.DataFrame()
    
    print(f"📥 Carregando dados de {min(max_files, len(parquet_files))} arquivos...")
    
    # Conectar ao DuckDB
    conn = duckdb.connect(':memory:')
    
    # Lista para armazenar DataFrames
    dfs = []
    
    for i, file_info in enumerate(parquet_files[:max_files]):
        try:
            print(f"  📄 Carregando {file_info['name']}...")
            
            # Usar DuckDB para ler Parquet direto da URL
            url = file_info['url']
            
            # Carregar amostra do arquivo
            query = f"""
            SELECT * FROM '{url}'
            USING SAMPLE {sample_size} ROWS
            """
            
            df = conn.execute(query).df()
            
            if not df.empty:
                df['arquivo_origem'] = file_info['name']
                dfs.append(df)
                print(f"    ✅ {len(df):,} registros carregados")
            
        except Exception as e:
            print(f"    ⚠️ Erro ao carregar {file_info['name']}: {e}")
            continue
    
    if dfs:
        # Combinar todos os DataFrames
        combined_df = pd.concat(dfs, ignore_index=True, sort=False)
        
        print(f"\n✅ Dataset carregado com sucesso!")
        print(f"📊 Total de registros: {len(combined_df):,}")
        print(f"📋 Colunas disponíveis: {len(combined_df.columns)}")
        
        return combined_df
    else:
        print("❌ Nenhum dado pôde ser carregado")
        return pd.DataFrame()

# Carregar amostra dos dados
print("🚀 Iniciando carregamento dos dados PNCP...")
df_contratos = load_sample_data(max_files=2, sample_size=50000)

if not df_contratos.empty:
    print(f"\n📈 Primeiras estatísticas:")
    print(f"  📅 Período: {df_contratos.get('dataAssinatura', pd.Series()).min()} a {df_contratos.get('dataAssinatura', pd.Series()).max()}")
    
    if 'valorGlobal' in df_contratos.columns:
        valor_total = df_contratos['valorGlobal'].sum() / 1_000_000
        print(f"  💰 Valor total (amostra): R$ {valor_total:,.0f} milhões")
    
    if 'uf_sigla' in df_contratos.columns or any('uf' in col.lower() for col in df_contratos.columns):
        uf_col = next((col for col in df_contratos.columns if 'uf' in col.lower()), None)
        if uf_col:
            ufs = df_contratos[uf_col].nunique()
            print(f"  🗺️ Estados representados: {ufs}")
else:
    print("⚠️ Continuando com dados de exemplo para demonstração...")
    # Criar dados de exemplo se não conseguir carregar do IA
    np.random.seed(42)
    df_contratos = pd.DataFrame({
        'numeroControlePncpCompra': [f'PNCP{i:06d}' for i in range(1000)],
        'valorGlobal': np.random.lognormal(10, 2, 1000),
        'dataAssinatura': pd.date_range('2024-01-01', periods=1000, freq='D'),
        'nomeRazaoSocialFornecedor': [f'Empresa {i%50}' for i in range(1000)],
        'orgao_razao_social': [f'Órgão {i%20}' for i in range(1000)],
        'uf_sigla': np.random.choice(['RO', 'SP', 'RJ', 'MG', 'RS'], 1000)
    })
    print(f"📊 Usando dados de exemplo: {len(df_contratos):,} registros")

## 🔍 Exploração Inicial dos Dados

Vamos entender a estrutura e qualidade dos dados carregados.

In [ ]:
# Visão geral do dataset
print("📋 ESTRUTURA DO DATASET")
print("=" * 50)
print(f"Registros: {len(df_contratos):,}")
print(f"Colunas: {len(df_contratos.columns)}")
print(f"Memória: {df_contratos.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

print("\n📊 PRINCIPAIS COLUNAS:")
print("-" * 30)
for col in df_contratos.columns[:15]:  # Primeiras 15 colunas
    dtype = df_contratos[col].dtype
    null_pct = (df_contratos[col].isnull().sum() / len(df_contratos)) * 100
    print(f"  {col[:30]:<30} | {str(dtype):<12} | {null_pct:5.1f}% nulos")

if len(df_contratos.columns) > 15:
    print(f"  ... e mais {len(df_contratos.columns) - 15} colunas")

# Mostrar amostra dos dados
print("\n🔍 AMOSTRA DOS DADOS:")
print("-" * 20)
display(df_contratos.head())

In [ ]:
# Análise de qualidade dos dados
print("🔍 ANÁLISE DE QUALIDADE DOS DADOS")
print("=" * 40)

# Campos essenciais para análise
essential_fields = [
    'numeroControlePncpCompra', 'valorGlobal', 'dataAssinatura',
    'nomeRazaoSocialFornecedor', 'orgao_razao_social'
]

# Verificar quais campos existem
available_fields = [field for field in essential_fields if field in df_contratos.columns]
missing_fields = [field for field in essential_fields if field not in df_contratos.columns]

print(f"✅ Campos disponíveis: {len(available_fields)}/{len(essential_fields)}")
for field in available_fields:
    completeness = (1 - df_contratos[field].isnull().mean()) * 100
    print(f"  📊 {field}: {completeness:.1f}% completo")

if missing_fields:
    print(f"\n⚠️ Campos ausentes: {missing_fields}")
    print("   Usando campos alternativos disponíveis...")

# Identificar colunas de valor
value_columns = [col for col in df_contratos.columns if 'valor' in col.lower()]
print(f"\n💰 Colunas de valor encontradas: {value_columns}")

# Identificar colunas de data
date_columns = [col for col in df_contratos.columns if 'data' in col.lower() or 'date' in col.lower()]
print(f"📅 Colunas de data encontradas: {date_columns}")

# Identificar colunas geográficas
geo_columns = [col for col in df_contratos.columns if any(geo in col.lower() for geo in ['uf', 'estado', 'municipio', 'cidade'])]
print(f"🗺️ Colunas geográficas encontradas: {geo_columns}")

## 🔧 Preparação dos Dados

Limpeza e transformação dos dados para análise.

In [ ]:
def prepare_data(df):
    """Prepara os dados para análise."""
    df_clean = df.copy()
    
    print("🔧 Preparando dados para análise...")
    
    # 1. Padronizar nomes de colunas importantes
    column_mapping = {}
    
    # Encontrar coluna de valor principal
    valor_col = None
    for col in ['valorGlobal', 'valor_global_brl', 'valor_total']:
        if col in df_clean.columns:
            valor_col = col
            break
    
    if valor_col:
        column_mapping[valor_col] = 'valor_contrato'
    
    # Encontrar coluna de data
    data_col = None
    for col in ['dataAssinatura', 'data_assinatura', 'data_contrato']:
        if col in df_clean.columns:
            data_col = col
            break
    
    if data_col:
        column_mapping[data_col] = 'data_assinatura'
    
    # Encontrar coluna de UF
    uf_col = None
    for col in ['uf_sigla', 'UF', 'estado_sigla']:
        if col in df_clean.columns:
            uf_col = col
            break
    
    if uf_col:
        column_mapping[uf_col] = 'uf'
    
    # Aplicar mapeamento
    df_clean = df_clean.rename(columns=column_mapping)
    
    # 2. Converter tipos de dados
    if 'valor_contrato' in df_clean.columns:
        df_clean['valor_contrato'] = pd.to_numeric(df_clean['valor_contrato'], errors='coerce')
        df_clean = df_clean[df_clean['valor_contrato'] > 0]  # Remover valores inválidos
    
    if 'data_assinatura' in df_clean.columns:
        df_clean['data_assinatura'] = pd.to_datetime(df_clean['data_assinatura'], errors='coerce')
        # Adicionar colunas derivadas
        df_clean['ano'] = df_clean['data_assinatura'].dt.year
        df_clean['mes'] = df_clean['data_assinatura'].dt.month
        df_clean['mes_ano'] = df_clean['data_assinatura'].dt.to_period('M').astype(str)
    
    # 3. Limpeza de campos texto
    text_columns = df_clean.select_dtypes(include=['object']).columns
    for col in text_columns:
        if df_clean[col].dtype == 'object':
            df_clean[col] = df_clean[col].astype(str).str.strip().str.upper()
            df_clean[col] = df_clean[col].replace(['NAN', 'NONE', 'NULL', ''], pd.NA)
    
    # 4. Criar categorias de valor
    if 'valor_contrato' in df_clean.columns:
        df_clean['categoria_valor'] = pd.cut(
            df_clean['valor_contrato'],
            bins=[0, 50000, 200000, 1000000, 10000000, float('inf')],
            labels=['Até R$ 50k', 'R$ 50k-200k', 'R$ 200k-1M', 'R$ 1M-10M', 'Acima R$ 10M'],
            include_lowest=True
        )
    
    print(f"✅ Dados preparados: {len(df_clean):,} registros válidos")
    
    return df_clean

# Preparar os dados
df_clean = prepare_data(df_contratos)

# Mostrar resumo dos dados limpos
print("\n📊 RESUMO DOS DADOS PREPARADOS:")
print(f"  📈 Registros válidos: {len(df_clean):,}")

if 'valor_contrato' in df_clean.columns:
    valor_total = df_clean['valor_contrato'].sum() / 1_000_000
    valor_medio = df_clean['valor_contrato'].mean()
    print(f"  💰 Valor total: R$ {valor_total:,.0f} milhões")
    print(f"  💰 Valor médio: R$ {valor_medio:,.0f}")

if 'data_assinatura' in df_clean.columns:
    data_min = df_clean['data_assinatura'].min()
    data_max = df_clean['data_assinatura'].max()
    print(f"  📅 Período: {data_min.strftime('%Y-%m-%d')} a {data_max.strftime('%Y-%m-%d')}")

if 'uf' in df_clean.columns:
    ufs_count = df_clean['uf'].nunique()
    print(f"  🗺️ Estados: {ufs_count}")

## 📈 Análise Temporal

Análise da evolução das contratações ao longo do tempo.

In [ ]:
if 'data_assinatura' in df_clean.columns and 'valor_contrato' in df_clean.columns:
    # Análise temporal - evolução mensal
    monthly_data = df_clean.groupby('mes_ano').agg({
        'valor_contrato': ['count', 'sum', 'mean'],
        'numeroControlePncpCompra': 'nunique'  # ou primeira coluna disponível
    }).round(2)
    
    monthly_data.columns = ['total_contratos', 'valor_total', 'valor_medio', 'contratos_unicos']
    monthly_data['valor_total_milhoes'] = monthly_data['valor_total'] / 1_000_000
    monthly_data = monthly_data.reset_index()
    
    # Gráfico de evolução temporal
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=('Volume de Contratos por Mês', 'Valor Total por Mês (R$ Milhões)'),
        vertical_spacing=0.12
    )
    
    # Volume de contratos
    fig.add_trace(
        go.Scatter(
            x=monthly_data['mes_ano'],
            y=monthly_data['total_contratos'],
            mode='lines+markers',
            name='Contratos',
            line=dict(color='#1f77b4', width=3),
            marker=dict(size=6)
        ),
        row=1, col=1
    )
    
    # Valor total
    fig.add_trace(
        go.Scatter(
            x=monthly_data['mes_ano'],
            y=monthly_data['valor_total_milhoes'],
            mode='lines+markers',
            name='Valor (R$ M)',
            line=dict(color='#ff7f0e', width=3),
            marker=dict(size=6),
            fill='tonexty'
        ),
        row=2, col=1
    )
    
    fig.update_layout(
        title="📈 Evolução Temporal das Contratações Públicas",
        height=600,
        showlegend=False
    )
    
    fig.update_xaxes(title_text="Mês/Ano", row=2, col=1)
    fig.update_yaxes(title_text="Número de Contratos", row=1, col=1)
    fig.update_yaxes(title_text="Valor (R$ Milhões)", row=2, col=1)
    
    fig.show()
    
    # Estatísticas temporais
    print("📊 ESTATÍSTICAS TEMPORAIS:")
    print(f"  📅 Meses analisados: {len(monthly_data)}")
    print(f"  📈 Média de contratos/mês: {monthly_data['total_contratos'].mean():,.0f}")
    print(f"  💰 Média de valor/mês: R$ {monthly_data['valor_total_milhoes'].mean():,.1f} milhões")
    
    # Mês com mais contratos
    mes_maior_volume = monthly_data.loc[monthly_data['total_contratos'].idxmax()]
    print(f"  🔝 Maior volume: {mes_maior_volume['mes_ano']} ({mes_maior_volume['total_contratos']:,} contratos)")
    
    # Mês com maior valor
    mes_maior_valor = monthly_data.loc[monthly_data['valor_total_milhoes'].idxmax()]
    print(f"  💎 Maior valor: {mes_maior_valor['mes_ano']} (R$ {mes_maior_valor['valor_total_milhoes']:,.1f} milhões)")
    
else:
    print("⚠️ Dados de data ou valor não disponíveis para análise temporal")

## 🗺️ Análise Geográfica

Distribuição das contratações por estado e região.

In [ ]:
if 'uf' in df_clean.columns and 'valor_contrato' in df_clean.columns:
    # Análise por UF
    uf_analysis = df_clean.groupby('uf').agg({
        'valor_contrato': ['count', 'sum', 'mean'],
        'numeroControlePncpCompra': 'nunique'
    }).round(2)
    
    uf_analysis.columns = ['total_contratos', 'valor_total', 'valor_medio', 'contratos_unicos']
    uf_analysis['valor_total_milhoes'] = uf_analysis['valor_total'] / 1_000_000
    uf_analysis = uf_analysis.reset_index().sort_values('valor_total_milhoes', ascending=False)
    
    # Mapa coroplético (usando códigos de UF)
    fig_map = px.choropleth(
        uf_analysis.head(15),  # Top 15 estados
        locations='uf',
        color='valor_total_milhoes',
        hover_name='uf',
        hover_data={'total_contratos': True, 'valor_medio': ':,.0f'},
        title="💰 Valor Total de Contratos por Estado (R$ Milhões)",
        color_continuous_scale="Viridis",
        locationmode="geojson-id"  # Usar se tiver GeoJSON
    )
    
    # Se não conseguir fazer mapa, fazer gráfico de barras
    try:
        fig_map.show()
    except:
        # Fallback para gráfico de barras
        fig_bar = px.bar(
            uf_analysis.head(15),
            x='uf',
            y='valor_total_milhoes',
            title="💰 Valor Total de Contratos por Estado (R$ Milhões)",
            labels={'valor_total_milhoes': 'Valor (R$ Milhões)', 'uf': 'Estado'},
            color='valor_total_milhoes',
            color_continuous_scale="Viridis"
        )
        fig_bar.update_layout(showlegend=False, height=500)
        fig_bar.show()
    
    # Gráfico de dispersão: Volume vs Valor
    fig_scatter = px.scatter(
        uf_analysis,
        x='total_contratos',
        y='valor_total_milhoes',
        size='valor_medio',
        text='uf',
        title="📊 Volume vs Valor Total por Estado",
        labels={
            'total_contratos': 'Número de Contratos',
            'valor_total_milhoes': 'Valor Total (R$ Milhões)',
            'valor_medio': 'Valor Médio'
        },
        color='valor_medio',
        color_continuous_scale="Plasma"
    )
    
    fig_scatter.update_traces(textposition='top center')
    fig_scatter.update_layout(height=500)
    fig_scatter.show()
    
    # Ranking dos estados
    print("🏆 RANKING DOS ESTADOS:")
    print("=" * 50)
    print("Top 10 por Valor Total:")
    for i, row in uf_analysis.head(10).iterrows():
        print(f"  {i+1:2d}. {row['uf']:2s} - R$ {row['valor_total_milhoes']:8.1f}M ({row['total_contratos']:,} contratos)")
    
else:
    print("⚠️ Dados geográficos não disponíveis para análise")

## 🏢 Análise de Fornecedores

Identificação dos principais fornecedores e padrões de concentração.

In [ ]:
# Encontrar coluna de fornecedor
fornecedor_col = None
for col in ['nomeRazaoSocialFornecedor', 'fornecedor_nome', 'supplier_name']:
    if col in df_clean.columns:
        fornecedor_col = col
        break

if fornecedor_col and 'valor_contrato' in df_clean.columns:
    # Análise dos fornecedores
    fornecedores = df_clean.groupby(fornecedor_col).agg({
        'valor_contrato': ['count', 'sum', 'mean'],
        'numeroControlePncpCompra': 'nunique'
    }).round(2)
    
    fornecedores.columns = ['total_contratos', 'valor_total', 'valor_medio', 'contratos_unicos']
    fornecedores['valor_total_milhoes'] = fornecedores['valor_total'] / 1_000_000
    
    # Calcular métricas de concentração
    fornecedores['participacao_valor'] = (fornecedores['valor_total'] / fornecedores['valor_total'].sum()) * 100
    fornecedores['participacao_volume'] = (fornecedores['total_contratos'] / fornecedores['total_contratos'].sum()) * 100
    
    fornecedores = fornecedores.reset_index().sort_values('valor_total_milhoes', ascending=False)
    
    # Top fornecedores por valor
    top_fornecedores = fornecedores.head(20)
    
    fig_fornecedores = px.bar(
        top_fornecedores.head(15),
        x='valor_total_milhoes',
        y=fornecedor_col,
        orientation='h',
        title="🏢 Top 15 Fornecedores por Valor Total",
        labels={'valor_total_milhoes': 'Valor Total (R$ Milhões)'},
        color='total_contratos',
        color_continuous_scale="Blues"
    )
    
    fig_fornecedores.update_layout(height=600, yaxis={'categoryorder': 'total ascending'})
    fig_fornecedores.show()
    
    # Análise de concentração
    print("📊 ANÁLISE DE CONCENTRAÇÃO DE FORNECEDORES:")
    print("=" * 50)
    
    # Top 10 concentram quanto?
    top10_valor = fornecedores.head(10)['participacao_valor'].sum()
    top10_volume = fornecedores.head(10)['participacao_volume'].sum()
    
    print(f"🔝 Top 10 fornecedores concentram:")
    print(f"   💰 {top10_valor:.1f}% do valor total")
    print(f"   📊 {top10_volume:.1f}% do volume de contratos")
    
    # Índice de concentração HHI (simplificado)
    hhi_valor = (fornecedores['participacao_valor'] ** 2).sum()
    print(f"\n📈 Índice HHI (concentração de valor): {hhi_valor:.0f}")
    if hhi_valor > 2500:
        print("   🔴 Alta concentração (>2500)")
    elif hhi_valor > 1500:
        print("   🟡 Concentração moderada (1500-2500)")
    else:
        print("   🟢 Baixa concentração (<1500)")
    
    # Fornecedores com muitos contratos (possível padrão suspeito)
    fornecedores_frequentes = fornecedores[fornecedores['total_contratos'] >= 10].sort_values('total_contratos', ascending=False)
    
    if len(fornecedores_frequentes) > 0:
        print(f"\n⚠️ Fornecedores com ≥10 contratos: {len(fornecedores_frequentes)}")
        print("   Top 5 mais frequentes:")
        for i, row in fornecedores_frequentes.head(5).iterrows():
            nome = row[fornecedor_col][:40] + "..." if len(row[fornecedor_col]) > 40 else row[fornecedor_col]
            print(f"   {i+1}. {nome} ({row['total_contratos']} contratos, R$ {row['valor_total_milhoes']:.1f}M)")
    
    # Distribuição de valores por fornecedor
    fig_dist = px.histogram(
        fornecedores,
        x='valor_total_milhoes',
        nbins=50,
        title="📈 Distribuição de Valores por Fornecedor",
        labels={'valor_total_milhoes': 'Valor Total (R$ Milhões)', 'count': 'Número de Fornecedores'}
    )
    fig_dist.update_layout(height=400)
    fig_dist.show()
    
else:
    print("⚠️ Dados de fornecedores não disponíveis para análise")

## 💰 Análise de Valores

Distribuição e padrões nos valores dos contratos.

In [ ]:
if 'valor_contrato' in df_clean.columns:
    # Estatísticas descritivas
    print("💰 ESTATÍSTICAS DE VALORES DOS CONTRATOS:")
    print("=" * 45)
    
    valores = df_clean['valor_contrato']
    
    print(f"📊 Total de contratos: {len(valores):,}")
    print(f"💎 Valor total: R$ {valores.sum():,.0f} ({valores.sum()/1_000_000:.1f} milhões)")
    print(f"📈 Valor médio: R$ {valores.mean():,.0f}")
    print(f"📊 Valor mediano: R$ {valores.median():,.0f}")
    print(f"🔝 Valor máximo: R$ {valores.max():,.0f}")
    print(f"🔻 Valor mínimo: R$ {valores.min():,.0f}")
    print(f"📏 Desvio padrão: R$ {valores.std():,.0f}")
    
    # Percentis
    percentis = [50, 75, 90, 95, 99]
    print(f"\n📊 Percentis:")
    for p in percentis:
        valor_p = valores.quantile(p/100)
        print(f"   P{p}: R$ {valor_p:,.0f}")
    
    # Distribuição por categorias de valor
    if 'categoria_valor' in df_clean.columns:
        cat_analysis = df_clean.groupby('categoria_valor').agg({
            'valor_contrato': ['count', 'sum'],
        })
        cat_analysis.columns = ['quantidade', 'valor_total']
        cat_analysis['percentual_quantidade'] = (cat_analysis['quantidade'] / cat_analysis['quantidade'].sum()) * 100
        cat_analysis['percentual_valor'] = (cat_analysis['valor_total'] / cat_analysis['valor_total'].sum()) * 100
        cat_analysis = cat_analysis.reset_index()
        
        # Gráfico de categorias
        fig_cat = make_subplots(
            rows=1, cols=2,
            subplot_titles=('Distribuição por Quantidade', 'Distribuição por Valor'),
            specs=[[{'type': 'domain'}, {'type': 'domain'}]]
        )
        
        fig_cat.add_trace(
            go.Pie(
                labels=cat_analysis['categoria_valor'],
                values=cat_analysis['percentual_quantidade'],
                name="Quantidade"
            ),
            row=1, col=1
        )
        
        fig_cat.add_trace(
            go.Pie(
                labels=cat_analysis['categoria_valor'],
                values=cat_analysis['percentual_valor'],
                name="Valor"
            ),
            row=1, col=2
        )
        
        fig_cat.update_layout(
            title="💰 Distribuição de Contratos por Categoria de Valor",
            height=500
        )
        fig_cat.show()
        
        print(f"\n📊 Distribuição por categorias:")
        for _, row in cat_analysis.iterrows():
            print(f"   {row['categoria_valor']}: {row['quantidade']:,} contratos ({row['percentual_quantidade']:.1f}%) - {row['percentual_valor']:.1f}% do valor")
    
    # Histograma de valores (log scale para melhor visualização)
    fig_hist = px.histogram(
        df_clean[df_clean['valor_contrato'] > 0],
        x='valor_contrato',
        nbins=50,
        title="📈 Distribuição de Valores dos Contratos (Escala Logarítmica)",
        labels={'valor_contrato': 'Valor do Contrato (R$)', 'count': 'Número de Contratos'}
    )
    fig_hist.update_xaxes(type="log")
    fig_hist.update_layout(height=400)
    fig_hist.show()
    
    # Contratos de alto valor (top 1%)
    threshold_top1 = valores.quantile(0.99)
    high_value_contracts = df_clean[df_clean['valor_contrato'] >= threshold_top1]
    
    print(f"\n🔝 CONTRATOS DE ALTO VALOR (Top 1%):")
    print(f"   Limiar: R$ {threshold_top1:,.0f}")
    print(f"   Quantidade: {len(high_value_contracts):,} contratos")
    print(f"   Valor total: R$ {high_value_contracts['valor_contrato'].sum():,.0f} ({high_value_contracts['valor_contrato'].sum()/1_000_000:.1f} milhões)")
    print(f"   Concentração: {(high_value_contracts['valor_contrato'].sum() / valores.sum()) * 100:.1f}% do valor total em 1% dos contratos")
    
else:
    print("⚠️ Dados de valores não disponíveis para análise")

## 🚨 Detecção de Padrões Suspeitos

Identificação de possíveis irregularidades usando algoritmos de detecção de anomalias.

In [ ]:
def detect_suspicious_patterns(df):
    """Detecta padrões suspeitos nos contratos."""
    
    print("🚨 DETECÇÃO DE PADRÕES SUSPEITOS:")
    print("=" * 40)
    
    suspicious_contracts = []
    
    if 'valor_contrato' in df.columns:
        # 1. Valores muito altos (outliers)
        q99 = df['valor_contrato'].quantile(0.99)
        high_value = df[df['valor_contrato'] > q99]
        
        print(f"🔍 1. Contratos de valor muito alto (>P99):")
        print(f"   📊 Threshold: R$ {q99:,.0f}")
        print(f"   🔢 Quantidade: {len(high_value):,} contratos")
        print(f"   💰 Valor total: R$ {high_value['valor_contrato'].sum()/1_000_000:.1f} milhões")
        
        suspicious_contracts.extend(high_value.index.tolist())
        
        # 2. Valores redondos suspeitos (múltiplos de 10.000, 50.000, 100.000)
        round_values = df[
            (df['valor_contrato'] % 10000 == 0) & 
            (df['valor_contrato'] >= 100000)
        ]
        
        print(f"\n🔍 2. Valores redondos suspeitos:")
        print(f"   🔢 Quantidade: {len(round_values):,} contratos")
        print(f"   📊 Percentual: {(len(round_values)/len(df))*100:.2f}% do total")
        
        suspicious_contracts.extend(round_values.index.tolist())
    
    # 3. Fornecedores com muitos contratos
    if fornecedor_col in df.columns:
        supplier_counts = df[fornecedor_col].value_counts()
        frequent_suppliers = supplier_counts[supplier_counts >= 5].index
        
        frequent_contracts = df[df[fornecedor_col].isin(frequent_suppliers)]
        
        print(f"\n🔍 3. Fornecedores muito frequentes (≥5 contratos):")
        print(f"   🏢 Fornecedores: {len(frequent_suppliers):,}")
        print(f"   🔢 Contratos: {len(frequent_contracts):,}")
        print(f"   📊 Percentual: {(len(frequent_contracts)/len(df))*100:.1f}% do total")
        
        suspicious_contracts.extend(frequent_contracts.index.tolist())
    
    # 4. Análise temporal - contratos em fins de semana
    if 'data_assinatura' in df.columns:
        df_temp = df.copy()
        df_temp['dia_semana'] = pd.to_datetime(df_temp['data_assinatura']).dt.dayofweek
        weekend_contracts = df_temp[df_temp['dia_semana'].isin([5, 6])]  # Sábado e Domingo
        
        print(f"\n🔍 4. Contratos assinados em fins de semana:")
        print(f"   🔢 Quantidade: {len(weekend_contracts):,} contratos")
        print(f"   📊 Percentual: {(len(weekend_contracts)/len(df))*100:.2f}% do total")
        
        suspicious_contracts.extend(weekend_contracts.index.tolist())
    
    # Consolidar contratos suspeitos (remover duplicatas)
    unique_suspicious = list(set(suspicious_contracts))
    suspicious_df = df.loc[unique_suspicious]
    
    print(f"\n📋 RESUMO DE SUSPEIÇÕES:")
    print(f"   🚨 Total de contratos suspeitos: {len(suspicious_df):,}")
    print(f"   📊 Percentual do dataset: {(len(suspicious_df)/len(df))*100:.1f}%")
    
    if 'valor_contrato' in suspicious_df.columns:
        suspicious_value = suspicious_df['valor_contrato'].sum()
        total_value = df['valor_contrato'].sum()
        print(f"   💰 Valor dos suspeitos: R$ {suspicious_value/1_000_000:.1f} milhões")
        print(f"   📈 Concentração: {(suspicious_value/total_value)*100:.1f}% do valor total")
    
    return suspicious_df

# Executar detecção de padrões suspeitos
if len(df_clean) > 0:
    suspicious_contracts = detect_suspicious_patterns(df_clean)
    
    # Mostrar alguns exemplos de contratos suspeitos
    if len(suspicious_contracts) > 0:
        print(f"\n🔍 EXEMPLOS DE CONTRATOS SUSPEITOS:")
        print("-" * 40)
        
        # Mostrar top 10 por valor
        if 'valor_contrato' in suspicious_contracts.columns:
            top_suspicious = suspicious_contracts.nlargest(10, 'valor_contrato')
            
            for i, (idx, row) in enumerate(top_suspicious.iterrows()):
                valor = row['valor_contrato']
                data = row.get('data_assinatura', 'N/A')
                fornecedor = row.get(fornecedor_col, 'N/A')[:50] + "..." if fornecedor_col and len(str(row.get(fornecedor_col, ''))) > 50 else row.get(fornecedor_col, 'N/A')
                
                print(f"  {i+1:2d}. R$ {valor:>12,.0f} | {data} | {fornecedor}")
        
        # Visualização da distribuição de suspeições
        if 'valor_contrato' in suspicious_contracts.columns:
            fig_suspicious = px.scatter(
                suspicious_contracts.sample(min(1000, len(suspicious_contracts))),
                x='data_assinatura' if 'data_assinatura' in suspicious_contracts.columns else range(len(suspicious_contracts)),
                y='valor_contrato',
                title="🚨 Contratos Suspeitos - Distribuição Temporal vs Valor",
                labels={'valor_contrato': 'Valor (R$)', 'data_assinatura': 'Data'},
                color='valor_contrato',
                color_continuous_scale="Reds",
                hover_data=[fornecedor_col] if fornecedor_col else None
            )
            fig_suspicious.update_layout(height=500)
            fig_suspicious.show()
    
else:
    print("⚠️ Dataset vazio - não é possível detectar padrões suspeitos")

## 💾 Exportação de Dados

Exporte os dados analisados para uso em outras ferramentas.

In [ ]:
def export_analysis_results():
    """Exporta os resultados da análise."""
    
    print("💾 EXPORTAÇÃO DE DADOS:")
    print("=" * 25)
    
    exports = []
    
    # 1. Dataset principal limpo
    if len(df_clean) > 0:
        df_clean.to_csv('contratos_pncp_limpos.csv', index=False)
        exports.append(f"📊 contratos_pncp_limpos.csv ({len(df_clean):,} registros)")
    
    # 2. Contratos suspeitos
    if 'suspicious_contracts' in locals() and len(suspicious_contracts) > 0:
        suspicious_contracts.to_csv('contratos_suspeitos.csv', index=False)
        exports.append(f"🚨 contratos_suspeitos.csv ({len(suspicious_contracts):,} registros)")
    
    # 3. Análise mensal
    if 'monthly_data' in locals():
        monthly_data.to_csv('analise_mensal.csv', index=False)
        exports.append(f"📅 analise_mensal.csv ({len(monthly_data)} meses)")
    
    # 4. Análise por UF
    if 'uf_analysis' in locals():
        uf_analysis.to_csv('analise_por_uf.csv', index=False)
        exports.append(f"🗺️ analise_por_uf.csv ({len(uf_analysis)} estados)")
    
    # 5. Top fornecedores
    if 'fornecedores' in locals():
        fornecedores.head(100).to_csv('top_fornecedores.csv', index=False)
        exports.append(f"🏢 top_fornecedores.csv (top 100)")
    
    # Mostrar arquivos exportados
    print("✅ Arquivos exportados:")
    for export in exports:
        print(f"   {export}")
    
    if not exports:
        print("⚠️ Nenhum dado disponível para exportação")
    
    # Instruções para download
    if exports:
        print(f"\n📥 Para baixar os arquivos no Colab:")
        print("   1. Clique no ícone de pasta 📁 no menu lateral")
        print("   2. Encontre os arquivos .csv gerados")
        print("   3. Clique com botão direito → Download")
        print("\n📋 Ou use o código abaixo para download automático:")
        print("```python")
        print("from google.colab import files")
        for export in exports:
            filename = export.split(' ')[1]
            print(f"files.download('{filename}')")
        print("```")

# Executar exportação
export_analysis_results()

## 📝 Conclusões e Próximos Passos

### 🎯 O que foi analisado:

1. **📊 Análise Temporal**: Evolução das contratações ao longo do tempo
2. **🗺️ Análise Geográfica**: Distribuição por estados e regiões
3. **🏢 Análise de Fornecedores**: Concentração e padrões de contratação
4. **💰 Análise de Valores**: Distribuição e outliers nos valores
5. **🚨 Detecção de Anomalias**: Identificação de padrões suspeitos

### 🔍 Possíveis análises adicionais:

- **Análise de Redes**: Relacionamentos entre órgãos e fornecedores
- **Análise de Texto**: Processamento dos objetos dos contratos
- **Machine Learning**: Modelos preditivos de risco
- **Análise Setorial**: Comparação entre diferentes tipos de contratação
- **Análise de Eficiência**: Comparação de preços por categoria

### 📚 Recursos adicionais:

- **Projeto Baliza**: [https://github.com/franklinbaldo/baliza](https://github.com/franklinbaldo/baliza)
- **PNCP**: [https://pncp.gov.br](https://pncp.gov.br)
- **Internet Archive**: Dados preservados permanentemente
- **Documentação**: Veja o repositório para mais detalhes

### 🤝 Como contribuir:

1. **Fork** o projeto Baliza no GitHub
2. **Melhore** os algoritmos de detecção
3. **Adicione** novas análises
4. **Compartilhe** suas descobertas
5. **Cite** este trabalho em pesquisas acadêmicas

---

**⭐ Se este notebook foi útil, dê uma estrela no [repositório Baliza](https://github.com/franklinbaldo/baliza)!**

**📧 Dúvidas ou sugestões? Abra uma [issue](https://github.com/franklinbaldo/baliza/issues) no GitHub.**

In [ ]:
# Resumo final da análise
print("🎯 RESUMO FINAL DA ANÁLISE")
print("=" * 30)

if len(df_clean) > 0:
    print(f"📊 Dataset analisado: {len(df_clean):,} contratos")
    
    if 'valor_contrato' in df_clean.columns:
        total_value = df_clean['valor_contrato'].sum() / 1_000_000
        print(f"💰 Valor total: R$ {total_value:,.1f} milhões")
    
    if 'data_assinatura' in df_clean.columns:
        date_range = f"{df_clean['data_assinatura'].min().strftime('%Y-%m')} a {df_clean['data_assinatura'].max().strftime('%Y-%m')}"
        print(f"📅 Período: {date_range}")
    
    if 'suspicious_contracts' in locals():
        suspicion_rate = (len(suspicious_contracts) / len(df_clean)) * 100
        print(f"🚨 Taxa de suspeição: {suspicion_rate:.1f}%")
    
    print(f"\n✅ Análise concluída com sucesso!")
    print(f"📁 Arquivos exportados disponíveis para download")
    print(f"🔄 Execute novamente para analisar dados mais recentes")

else:
    print("⚠️ Análise não pôde ser concluída - dados não disponíveis")
    print("🔄 Tente executar novamente ou verifique a conexão")

print("\n🚀 Obrigado por usar o Baliza Analytics!")
print("⭐ Considere dar uma estrela no projeto: https://github.com/franklinbaldo/baliza")